#### Global Variables

In [ ]:
drive_dir = "/content/drive/MyDrive/UNICAP/TCC/research"
work_dir = f"{drive_dir}/results"
data_dir = f"{drive_dir}/datasets"
checkpoints_dir = f"{drive_dir}/checkpoints"
datasets = ["heridal"]
checkpoints = []

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu May 25 23:51:19 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


## Setup Environment

In [ ]:
from google.colab import drive
import shutil, errno
import fileinput
import json
import re
import os

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/AFKaro/SARDeep.git

Cloning into 'SARDeep'...
remote: Enumerating objects: 835, done.
remote: Counting objects: 100% (93/93), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 835 (delta 41), reused 55 (delta 22), pack-reused 742
Receiving objects: 100% (835/835), 157.42 KiB | 597.00 KiB/s, done.
Resolving deltas: 100% (450/450), done.


In [ ]:
!pip install optimum[neural-compressor]
!pip install tqdm
!pip install scipy
!pip install terminaltables
!pip install six
!pip install pycocotools
!pip install tensorboard
!pip install opencv-python
!pip install python-dotenv
!pascal-voc-writer==0.1.4
!sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 112.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 9.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 115.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/bin/bash: pascal-voc-writer==0.1.4: command not found
/bin/bash: sklearn: command not found


In [ ]:
!apt update
!apt install ffmpeg libsm6 libxext6 git ninja-build libglib2.0-0 libsm6 libxrender-dev libxext6
!apt clean
!rm -rf /var/lib/apt/lists/*

Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:11 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:12 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal/main Sources [2,581 kB]
Get:13 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal/main amd64 Packages [1,218

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117
!pip install openmim
!pip install mmcv-full==1.7.0 -f https://download.openmmlab.com/mmcv/dist/cu117/torch1.13/index.html
!cd /content/SARDeep/src && git clone -b 2.x --single-branch https://github.com/open-mmlab/mmdetection.git
%cd /content/SARDeep/src/mmdetection
!pip install -e .

Looking in indexes: https://download.pytorch.org/whl/cu117, https://us-python.pkg.dev/colab-wheels/public/simple/
INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 74.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 115.8 MB/s eta 0:00:00
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.0.2+cu118
    Uninstalling torchaudio-2.0.2+cu118:
      Successfully uninstalled torchaudio-2.0.2+cu118
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 kB 2.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-p

In [ ]:
def copy(src, dst):
    if not os.path.isdir(dst):
      try:
          shutil.copytree(src, dst)
      except OSError as exc: # python >2.5
          if exc.errno in (errno.ENOTDIR, errno.EINVAL):
              shutil.copy(src, dst)
          else: raise

for dataset in datasets:
  copy(f"{data_dir}/{dataset}", f"/content/SARDeep/src/mmdetection/data/{dataset}")

copy(checkpoints_dir, f"/content/SARDeep/src/infra/checkpoints")

In [ ]:
for checkpoint in checkpoints:
  !python /content/SARDeep/src/infra/scripts/download_weights.py --weights $checkpoint

In [ ]:
import dotenv
import os

dotenv_file = dotenv.find_dotenv()
dotenv.load_dotenv(dotenv_file)

env_vars = ("CHECKPOINTS_PATH", "RESOURCE_PATH", "CONFIG_PATH", "WORK_DIR", 
            "DATA_ROOT", "DEVICE", "DATA_PATH", "BASE_DIR")

os.environ["CHECKPOINTS_PATH"] = "../infra/checkpoints"
os.environ["RESOURCE_PATH"] = "../../resource"
os.environ["CONFIG_PATH"] = "../../src/infra/configs"
os.environ["WORK_DIR"] = f"../../../../{work_dir}"
os.environ["DATA_ROOT"] = "../mmdetection/data"
os.environ["DEVICE"] = "cuda"
os.environ["DATA_PATH"] = "/content/drive/MyDrive/UNICAP/TCC/research/datasets"
os.environ["BASE_DIR"] = "/content/SARDeep/src"

for env in env_vars:
  dotenv.set_key(dotenv_file, env, os.environ[env])


In [ ]:
!python /content/SARDeep/src/infra/scripts/config_classes.py

/content/drive/MyDrive/UNICAP/TCC/research/datasets sard
Classes: ['person']


In [ ]:
def update_config(model_name, train_data, test_data, fold, max_epochs, lr, momentum, interval):
  with open("/content/SARDeep/resource/pipeline.json", "r") as jsonFile:
    data = json.load(jsonFile)
  
  with open("/content/SARDeep/resource/models.json", "r") as jsonFile:
    data_models = json.load(jsonFile)

  data["models"] = [model_name]
  data["datasets"]["dataset_train"] = train_data
  data["datasets"]["dataset_test"] = test_data
  data["datasets"]["fold"] = f"fold{fold}"

  data["priority_steps"][0]["dataset_train"] = train_data
  data["priority_steps"][0]["dataset_test"] = test_data

  for idx, models in enumerate(data_models["models"]):
    if f"""{models["name"]}{models["version"]}""" == model_name:
      data_models["models"][idx]["runner"]["max_epochs"] = max_epochs
      data_models["models"][idx]["optimizer"]["lr"] = lr
      data_models["models"][idx]["optimizer"]["momentum"] = momentum
      data_models["models"][idx]["optimizer"]["interval"] = interval
  
  print(data_models)

  with open("/content/SARDeep/resource/pipeline.json", "w") as jsonFile:
      json.dump(data, jsonFile)
  
  with open("/content/SARDeep/resource/models.json", "w") as jsonFile:
      json.dump(data_models, jsonFile)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Pipeline

In [ ]:
model_name = "yolox_l" #@param {type:"string"}
step = "Train" #@param ["All", "Train", "Quantization", "Test", "Eval"]
#@markdown ---
lr = 0.002 #@param {type:"number"}
momentum = 0.9 #@param {type:"number"}
interval = 20 #@param {type:"integer"}
max_epochs = 100 #@param {type:"integer"}
#@markdown ---
train_data = "sard" #@param ["sard", "heridal", "seadronesee"]
fold = 1 #@param {type:"integer"}
test_data = "sard" #@param ["sard", "heridal", "seadronesee"]


update_config(model_name, train_data, test_data, fold, max_epochs, lr, momentum, interval)


{'models': [{'name': 'faster_rcnn', 'version': '', 'optimizer': {'lr': 0.01, 'momentum': 0.9, 'interval': 5}, 'runner': {'max_epochs': 60}, 'fine_tune': {'name': 'faster_rcnn_r50_fpn_1x_coco', 'load_from': 'CHECKPOINTS_PATH/faster_rcnn_r50_fpn_1x_coco.pth'}}, {'name': 'yolo', 'version': 'v3', 'optimizer': {'lr': 0.001, 'momentum': 0.9, 'interval': 5}, 'runner': {'max_epochs': 100}, 'fine_tune': {'name': 'yolov3_d53_fp16_mstrain-608_273e_coco', 'load_from': 'CHECKPOINTS_PATH/yolov3_d53_fp16_mstrain-608_273e_coco.pth'}}, {'name': 'yolo', 'version': 'x', 'optimizer': {'lr': 0.01, 'momentum': 0.9, 'interval': 5}, 'runner': {'max_epochs': 30}, 'fine_tune': {'name': 'yolox_x_8x8_300e_coco', 'load_from': 'CHECKPOINTS_PATH/yolox_x_8x8_300e_coco.pth'}}, {'name': 'yolo', 'version': 'x_tiny', 'optimizer': {'lr': 0.01, 'momentum': 0.9, 'interval': 20}, 'runner': {'max_epochs': 200}, 'fine_tune': {'name': 'yolox_tiny_8x8_300e_coco', 'load_from': 'CHECKPOINTS_PATH/yolox_tiny_8x8_300e_coco.pth'}}, {'

In [ ]:
!python /content/SARDeep/src/main.py --step $step